In [1]:
import numpy as np
import rescomp

import matplotlib.pyplot as plt

In [ ]:
# best one sofar
def get_lypunov_exponents_minimal(data_creation_function, tau, T, N, dt, eps, m, starting_point=None, iteration_process=False):
    
    # setting up
    tau_timesteps = int(tau/dt)
    T_timesteps = int(T/dt)
    
    if tau_timesteps == 0:
        if starting_point is None:
            print("ERROR: starting_point is None, and tau is 0")
            return
        initial_condition = starting_point
    else:     
        initial_condition = data_creation_function(tau_timesteps, dt, starting_point=starting_point)[-1] # discard transient states
    state_dim = initial_condition.size
    if m>state_dim:
        print("ERROR: m > state_dim")
        return
    
    # choose initial orthogonal directions:
    Q = np.eye(state_dim, m)
    
    # Space to save u_js:
    U = np.zeros((state_dim, N+1))
    U[:, 0] = initial_condition
    
    PsiQ = np.zeros(Q.shape)
    
    R_diags = np.zeros((N, m))
    
    for j in range(1, N+1): # for all time intervals
        u_jm1 = U[:, j-1]
        u_j = data_creation_function(T_timesteps, dt, starting_point = u_jm1)[-1]
        U[:, j] = u_j
        for i in range(m): # for all dimensions
            q_i = Q[:, i] # the (j-1)th Q
            u_jm1_per = u_jm1 + eps*q_i
            w_i_j = data_creation_function(T_timesteps, dt, starting_point = u_jm1_per)[-1]
            PsiQ[:, i] = (w_i_j - u_j)/eps # approximation of Psi(t_j, t_jm1)q_i(j_m1)
        
        Q, R = np.linalg.qr(PsiQ) # the new Q (the jth)

        for i in range(m):
            R_diags[j-1, i] = R[i, i]
    lyapunov_exp = np.sum(np.log(np.abs(R_diags))/(N*T), axis = 0)
    if iteration_process:
        return lyapunov_exp, np.cumsum(np.log(np.abs(R_diags))/(N*T), axis = 0)
    return lyapunov_exp

# best one sofar
def get_lypunov_exponents_minimal(data_creation_function, tau, T, N, dt, eps, m, starting_point=None, iteration_process=False):
    
    # setting up
    tau_timesteps = int(tau/dt)
    T_timesteps = int(T/dt)
    
    if tau_timesteps == 0:
        if starting_point is None:
            print("ERROR: starting_point is None, and tau is 0")
            return
        initial_condition = starting_point
    else:     
        initial_condition = data_creation_function(tau_timesteps, dt, starting_point=starting_point)[-1] # discard transient states
    state_dim = initial_condition.size
    if m>state_dim:
        print("ERROR: m > state_dim")
        return
    
    # choose initial orthogonal directions:
    Q = np.eye(state_dim, m)
    
    # Space to save u_js:
    U = np.zeros((state_dim, N+1))
    U[:, 0] = initial_condition
    
    PsiQ = np.zeros(Q.shape)
    
    R_diags = np.zeros((N, m))
    
    for j in range(1, N+1): # for all time intervals
        u_jm1 = U[:, j-1]
        u_j = data_creation_function(T_timesteps, dt, starting_point = u_jm1)[-1]
        U[:, j] = u_j
        for i in range(m): # for all dimensions
            q_i = Q[:, i] # the (j-1)th Q
            u_jm1_per = u_jm1 + eps*q_i
            w_i_j = data_creation_function(T_timesteps, dt, starting_point = u_jm1_per)[-1]
            PsiQ[:, i] = (w_i_j - u_j)/eps # approximation of Psi(t_j, t_jm1)q_i(j_m1)
        
        Q, R = np.linalg.qr(PsiQ) # the new Q (the jth)

        for i in range(m):
            R_diags[j-1, i] = R[i, i]
    lyapunov_exp = np.sum(np.log(np.abs(R_diags))/(N*T), axis = 0)
    if iteration_process:
        return lyapunov_exp, np.cumsum(np.log(np.abs(R_diags))/(N*T), axis = 0)
    return lyapunov_exp

In [ ]:
# Joshkas Function
def equation_based_lyapunov_spectrum_discrete(f, Jacobian, starting_point,
                                              nr_steps=3000, dt=1.,
                                              return_convergence=False):
    """ Calculates the lyapunov spectrum of a discrete dynamical system
    with x_n+1 = f(x_n) using a standard QR-based algorithm.

    Based on equations not data.

    Measure for chaotic behaviour in the system.

    Important characteristic to compare attractors.

    Args:
        f (function): mapping with x_n+1 = f(x_n)
        Jacobian (function): Jacobian of f , takes x as argument
        starting_point (np.ndarray): inintial condition of iteration
        nr_steps (int): number of iteration steps
        dt (float): time scale of a step
        return_convergence (bool): if True returns the development of the
                estimate for the lyapunov spectrum over time in steps of 100
                iterations.

    Returns:
        np.ndarray_or_tuple : lyapunov spectrum if return_convergence is False,
                                tuple of final lyapunov spectrum and development
                                of lyapunov spectrum if return_convergence is
                                True

     """
    x = starting_point

    # jac = Jacobian(x)
    # jac_sparse = scipy.sparse.csr_matrix(Jacobian(x))

    Q, R = np.linalg.qr(Jacobian(x))

    s = []
    lces = []

    for n in range(nr_steps):
        x = f(x)
        Q, R = np.linalg.qr(np.matmul(Jacobian(x), Q))

        s.append(np.array([R[i, i] for i in range(len(R))]))

        if return_convergence:
            lya = np.sum(np.log(np.abs(s)), axis=0) / ((n + 1) * dt)
            lces.append(lya)

    lya = np.sum(np.log(np.abs(s)), axis=0) / (nr_steps * dt)

    if return_convergence:
        return lya, np.array(lces)
    else:
        return lya
    


In [2]:
# KS system:
dimensions = 64
system_size = 35
def data_creation_function_KS(time_steps, dt, starting_point=None):
    sim_data = rescomp.simulate_trajectory(
            sys_flag='kuramoto_sivashinsky', dimensions=dimensions, system_size=system_size, dt=dt,
            time_steps=time_steps, starting_point = starting_point)
    return sim_data

# Lorenz system
def data_creation_function_Lorenz(time_steps, dt, starting_point=None):
    return  rescomp.simulate_trajectory(
            sys_flag='lorenz', dt=dt, time_steps=time_steps,
            starting_point=starting_point)

In [ ]:
# Jacobian Lorenz:
def Jacobian_Lorenz(x, dt, sigma=10, rho=28, beta=8/3):
    '''
    The jacobian of the iterator of the Lorenz system based on the simple forward Euler
    '''
    return np.eye(3) + dt*np.array([[-sigma, sigma, 0],
                                    [-x[2] + rho, -1, x[0]],
                                    [x[1], x[0], -beta]])


Jacobian_Lorenz([0,0,0], dt= 0.1)

In [6]:
starting_point = np.array([-14.03020521, -20.88693127, 25.53545])

dt = 0.1
N = 2000
tau = 100
T = 2
eps = 1e-6
f = lambda x: data_creation_function_Lorenz(time_steps=2, dt=dt, starting_point=x)[-1]


lyapunov_exp, lyapunov_exp_convergence = rescomp.measures.iterator_based_lyapunov_spectrum(f, starting_point, T=T, 
                                                                                           tau=tau,
                                                  eps=eps, nr_steps=N, dt=dt, return_convergence=True)

KeyboardInterrupt: 

In [ ]:
print(lyapunov_exp)
plt.figure(figsize=(10,5))
for iteration in lyapunov_exp_convergence.T:
    plt.plot(iteration)
plt.grid()

In [8]:
get_lypunov_exponents_Lorenz(data_creation_function, tau=tau, T=T,
                                              N=N, dt=dt, eps=eps, m=m, starting_point=starting_point,
                                              iteration_process=True)

NameError: name 'get_lypunov_exponents_Lorenz' is not defined

# Lorenz:

In [ ]:
starting_point = np.array([-14.03020521, -20.88693127, 25.53545])

dt = 0.1
N = 20000
tau = 100

# For predictor based calculation:
T = 1
eps = 1e-8
m = 3
iteration_process=True

# For Joshas code:
f = lambda x: data_creation_function_Lorenz(time_steps=1, dt=dt, starting_point=x)[-1]
Jacobian = lambda x: Jacobian_Lorenz(x, dt=dt)
return_convergence=True

In [ ]:
# Only based on predictor function x_n+1 = F(x)

data_creation_function = data_creation_function_Lorenz
get_lypunov_exponents = get_lypunov_exponents_minimal

lyapunovs, iterations = get_lypunov_exponents(data_creation_function, tau=tau, T=T,
                                              N=N, dt=dt, eps=eps, m=m, starting_point=starting_point,
                                              iteration_process=True)

print(lyapunovs)
plt.figure(figsize=(10,5))
for iteration in iterations.T:
    plt.plot(iteration)
plt.grid()

In [ ]:
# Joshkas code:
lyapunovs, iterations = equation_based_lyapunov_spectrum_discrete(f, Jacobian, starting_point,
                                              nr_steps=N, dt=dt,
                                              return_convergence=return_convergence)
print(lyapunovs)
plt.figure(figsize=(10,5))
for iteration in iterations.T:
    plt.plot(iteration)
plt.grid()

# KS:

In [ ]:
# diag function

data_creation_function = data_creation_function_KS
get_lypunov_exponents = get_lypunov_exponents_minimal

iterations = get_lypunov_exponents(data_creation_function, tau=2000, T=2, N=100, dt=0.1, 
                                   eps=1e-6, m=10, iteration_process=True)


plt.figure(figsize=(10,5))
plt.plot(iterations)
plt.grid()
print(iterations[-1])

In [ ]:
# abs function

data_creation_function = data_creation_function_KS
get_lypunov_exponents = get_lypunov_exponents_minimal

iterations = get_lypunov_exponents(data_creation_function, tau=5, T=0.1, N=1000, dt=0.01, 
                                   eps=1e-6, m=2, iteration_process=True)
plt.figure(figsize=(10,5))
plt.plot(iterations)
plt.grid()
print(iterations[-1])

In [ ]:
# Matrix tests to understand the axis conventions in numpy:
# >>> first index is the row second index is the column
# >>> first index moves in most nested list 

A = np.zeros((2,2))
B = np.array([[4,2], [1,3]])

A[0,0] = 1
A[0,1] = 2
A[1,0] = 3
A[1,1] = 4
print(A)
print(B)

In [ ]:
A[0]

In [ ]:
A.dot(B)

In [ ]:
get_lypunov_exponents_minimal.__code__.co_argcount

In [ ]:
data_creation_function_Lorenz(time_steps=5, dt=0.1, starting_point=None).shape

In [ ]:
np.arange(1, 4+1)